<a href="https://colab.research.google.com/github/anemtinova/urfu_observing_proposal/blob/main/urfu_observing_proposal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Предположим, что мы хотим наблюдать транзит [экзопланеты TOI-4074.01](https://exoplanetarchive.ipac.caltech.edu/overview/TIC%20628103717#planet_TOI-4074-01_collapsible) на телескопе **RoboPhot** в Коуровской астрономической обсерватории.

Первое, в чем стоит убедиться: видимая звездная величина в фильтрах r и i должна быть меньше предельной звездной величины телескопа. Звездную величину легко проверить по вышеуказанной ссылке, а также убедиться, что она совпадает со звездной величиной в каталоге SDSS (DR12):

> Устанавливаем **astroquery** для отправки запросов в базы данных:



In [ ]:
pip install astroquery

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 910.2/910.2 kB 24.8 MB/s eta 0:00:00


> Вводим ICRS координаты звезды:



In [ ]:
star_ra = "01h57m41.61s"
star_dec = "+63d54m16.52s"

> Выполняем поиск звезд внутри окружности с радиусом 2 угловые секунды:



In [ ]:
from astropy.coordinates import SkyCoord, Angle

In [ ]:
from astroquery.vizier import Vizier

star_coord = [Angle(star_ra), Angle(star_dec)]
star_ra_dec = SkyCoord(star_coord[0], star_coord[1], frame = 'icrs')
result = Vizier.query_region(star_ra_dec, radius = Angle("2s"), catalog = "SDSS12")
print(result[0]['rmag'])
print(result[0]['imag'])

 rmag 
 mag  
------
14.564
 imag 
 mag  
------
14.071


14 звездная величина будет отлично наблюдаться на RoboPhot. Осталось определиться с датой и временем наблюдений.

Для этого на том же сайте находим **Transit Prediction Service** и переходим на [страницу NASA EXOPLANET ARCHIVE](https://exoplanetarchive.ipac.caltech.edu/cgi-bin/TransitView/nph-visibletbls?dataset=transits&sname=TOI-4074.01&getParams=1), где нам нужно указать **Observer Location (Custom Lat/Lon в десятичных градусах)**.

> Координаты Коуровской астрономической обсерватории: Latitude = 57$^\circ$ 02' 11" N, Longitude = 59$^\circ$ 32' 30" E. Воспользуемся объектом **Angle** для перевода градусов, минут и секунд в градусы.



In [ ]:
lat = Angle("57d02m11s")
print(lat.degree)
lon = Angle("59d32m30s")
print(lon.degree)

57.03638888888889
59.541666666666664


Выбираем **Next Event**, если хотим получить дату и время ближайшего транзита. Отправляем запрос и получаем таблицу со следующими полезными данными: продолжительность транзита — 3.2 часа, дата и время середины транзита — 12 марта 2024 года в 21:39 UT. Учитывая общую продолжительность наблюдений в 4 часа, можно определить, что начать наблюдение транзита можно в 19:39 UT.

Оценим общие условия наблюдений (например, в момент середины транзита): фазу Луны, угловое расстояние от звезды до Луны на небесной сфере и высоту звезды над горизонтом.

> Устанавливаем **ephem** (сокращение от эфемерида), использующийся для стандартных астрономических вычислений:



In [ ]:
pip install ephem

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 7.1 MB/s eta 0:00:00


In [ ]:
import ephem

moon = ephem.Moon()
observer = ephem.Observer()
observer.date = "2024/3/12 21:39:00"

> Находим фазу Луны **moon_phase** — процент освещенной части Луны (не зависит от положения наблюдателя). Полнолуние мешает наблюдениям из-за яркого света Луны, который может затмить слабые объекты на небе и ухудшить качество изображений. Наблюдения лучше проводить во время новолуния.



In [ ]:
moon.compute(observer.date)
print(moon.moon_phase)

0.09384122978749931


> Экваториальные координаты Луны в момент середины транзита:


In [ ]:
print('%s %s' % (moon.a_ra, moon.a_dec))

1:42:07.45 11:48:39.0


> Используя **SkyCoord.separation**, можно найти угловое расстояние между двумя точками (звездой и Луной) на небесной сфере:



In [ ]:
moon_ra_dec = SkyCoord("1h42m07.45s", "11d48m39.0s", frame = 'icrs')
sep = star_ra_dec.separation(moon_ra_dec)
print(sep)

52d09m57.01693238s


Таким образом, Луна не помешает наблюдениям, так как ее освещенность составляет всего 9% и она находится на достаточно большом угловом расстоянии.

> Звезда в момент середины транзита будет находиться на высоте:



In [ ]:
from astropy import units as u
from astropy.coordinates import EarthLocation, AltAz
from astropy.time import Time

location = EarthLocation(lat = lat.value, lon = lon.value, height = 290 * u.m)
obs_time = Time("2024-3-12 21:39")

alt_az_frame = AltAz(location = location, obstime = obs_time)
star_alt_az = star_ra_dec.transform_to(alt_az_frame)

print(star_alt_az.alt)

31d34m21.8397726s


В темное время суток звезда значительно выше горизонта, что позволит попытаться наблюдать транзит в Коуровской обсерватории.

